# Shift maliciousness (smallNORB)

**Author**: Maleakhi A. Wijaya  
**Description**: Inspect the maliciousness of different type and intensity of shifts.

In [1]:
# Load utilities functions
%run ../../scripts/constants.py
%run ../../scripts/smallnorb_utils.py
%run ../../scripts/shift_applicator.py
%run ../../scripts/shift_dimensionality_reductor.py
%run ../../scripts/experiment_utils.py
%run ../../scripts/shift_statistical_test.py
from sklearn.metrics import accuracy_score

In [2]:
## Random seed
SEED = 20
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load dataset

In [3]:
files_dir = "../../data"
# index 0 = image category
X_train, X_test, y_train, y_test, c_train, c_test = train_test_split_smallnorb(files_dir, DatasetTask.Task1, 
                                                                               train_size=0.75, class_index=0,
                                                                              resize_size=64)
n_classes = 5
concept_names = ['category', 'instance', 'elevation', 'azimuth', 'lighting']
concept_values = get_latent_sizes()

# Split training into validation set as well 
X_valid, X_train = X_train[:10000], X_train[10000:]
y_valid, y_train = y_train[:10000], y_train[10000:]
c_valid, c_train = c_train[:10000], c_train[10000:]

Training samples: 36450
Testing samples: 12150


In [5]:
# Load adversarial samples (we will need this for adversarial
# shift).
adv_samples = np.load("../../data/adversarial_samples/X_adversarial_smallnorb.npy")

In [6]:
# Reshape to appropriate shift input
# It is noteworthy that for efficiency, we represent the images as only 2 dimension
# when we preprocessing (number of instances/ batch size * flatten size).
# When visualising back the image, we need to reshape it back to the original dimension
ORIGINAL_SHAPE = X_test.shape[1:] # constant hold the image original shape
X_test_flatten = deepcopy(X_test.reshape(X_test.shape[0], -1))
X_train_flatten = deepcopy(X_train.reshape(X_train.shape[0], -1))
X_valid_flatten = deepcopy(X_valid.reshape(X_valid.shape[0], -1))

In [4]:
# Load models
path = "../../models/end_to_end_smallnorb_task1"
end_to_end_model = tf.keras.models.load_model(path)

path = "../../models/multitask_smallnorb"
itc_model = tf.keras.models.load_model(path)

In [8]:
# Evaluate model (FFNN)
y_pred = end_to_end_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

Accuracy: 0.9983539094650206


In [7]:
# Evaluate model (concept bottleneck model)
for i, pred in enumerate(itc_model.predict(X_test)):
    print("*"*20, f"Model: {SMALLNORB_CONCEPT_NAMES[i]}", "*"*20)
    c_truth = c_test[:, i]
    c_pred = np.argmax(pred, axis=1)
    
    print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")

******************** Model: category ********************
Accuracy: 0.9966255144032922
******************** Model: instance ********************
Accuracy: 0.8983539094650206
******************** Model: elevation ********************
Accuracy: 0.5077366255144032
******************** Model: azimuth ********************
Accuracy: 0.8267489711934156
******************** Model: lighting ********************
Accuracy: 0.9203292181069959


## Gaussian shift

In [9]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_gaussian = []

for shift_intensity in shift_intensities:
    X_shifted = deepcopy(X_test_flatten)
    X_shifted, _ = apply_gaussian_shift(X_shifted, y_test, shift_intensity, 1.0)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_gaussian.append(X_shifted)

In [10]:
# Print accuracy score (end-to-end)
for x, intensity_str in zip(X_gaussian, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

******************** small ********************
Accuracy: 0.9983539094650206
******************** medium ********************
Accuracy: 0.9980246913580247
******************** large ********************
Accuracy: 0.5497942386831276


In [11]:
# Print accuracy score (cbm)
for x, intensity_str in zip(X_gaussian, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SMALLNORB_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c_test[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: category **********
Accuracy: 0.9967901234567901
********** Model: instance **********
Accuracy: 0.8974485596707819
********** Model: elevation **********
Accuracy: 0.508559670781893
********** Model: azimuth **********
Accuracy: 0.8263374485596707
********** Model: lighting **********
Accuracy: 0.9209876543209876

******************** medium ********************
********** Model: category **********
Accuracy: 0.9838683127572017
********** Model: instance **********
Accuracy: 0.8469958847736625
********** Model: elevation **********
Accuracy: 0.4448559670781893
********** Model: azimuth **********
Accuracy: 0.7778600823045267
********** Model: lighting **********
Accuracy: 0.9132510288065844

******************** large ********************
********** Model: category **********
Accuracy: 0.26386831275720163
********** Model: instance **********
Accuracy: 0.15613168724279836
********** Model: elevation **********
Accuracy:

## Knockout shift

In [12]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_ko = []
y_ko = []
c_ko = []

for shift_intensity in shift_intensities:
    X_shifted, y_shifted, c_shifted = deepcopy(X_test_flatten), deepcopy(y_test), deepcopy(c_test)
    X_shifted, y_shifted, c_shifted = apply_ko_shift(X_shifted, y_shifted, c_shifted, shift_intensity, cl=MAJORITY)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_ko.append(X_shifted)
    y_ko.append(y_shifted)
    c_ko.append(c_shifted)

In [13]:
# Print accuracy score (end-to-end)
for x, y, c, intensity_str in zip(X_ko, y_ko, c_ko, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y, y_pred)}")

******************** small ********************
Accuracy: 0.998320174701831
******************** medium ********************
Accuracy: 0.9982616651418116
******************** large ********************
Accuracy: 0.9981464318813716


In [15]:
# Print accuracy score (cbm)
for x, y, c, intensity_str in zip(X_ko, y_ko, c_ko, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SMALLNORB_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: category **********
Accuracy: 0.9965563581387535
********** Model: instance **********
Accuracy: 0.8988745170502268
********** Model: elevation **********
Accuracy: 0.509323030404838
********** Model: azimuth **********
Accuracy: 0.8268940030236855
********** Model: lighting **********
Accuracy: 0.9202082983369729

******************** medium ********************
********** Model: category **********
Accuracy: 0.996523330283623
********** Model: instance **********
Accuracy: 0.8983531564501372
********** Model: elevation **********
Accuracy: 0.5109789569990851
********** Model: azimuth **********
Accuracy: 0.829551692589204
********** Model: lighting **********
Accuracy: 0.9215919487648674

******************** large ********************
********** Model: category **********
Accuracy: 0.9962928637627433
********** Model: instance **********
Accuracy: 0.8991864895479353
********** Model: elevation **********
Accuracy: 0.5

## Concept shift

In [16]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_cs = []
y_cs = []
c_cs = []

for shift_intensity in shift_intensities:
    X_shifted, y_shifted, c_shifted = deepcopy(X_test_flatten), deepcopy(y_test), deepcopy(c_test)
    X_shifted, y_shifted, c_shifted = apply_concept_shift(X_shifted, y_shifted, c_shifted, 2, shift_intensity, cl=MAJORITY)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_cs.append(X_shifted)
    y_cs.append(y_shifted)
    c_cs.append(c_shifted)

In [17]:
# Print accuracy score (end-to-end)
for x, y, c, intensity_str in zip(X_cs, y_cs, c_cs, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y, y_pred)}")

******************** small ********************
Accuracy: 0.9983347210657785
******************** medium ********************
Accuracy: 0.9987775061124694
******************** large ********************
Accuracy: 0.9988841361353915


In [19]:
# Print accuracy score (cbm)
for x, y, c, intensity_str in zip(X_cs, y_cs, c_cs, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SMALLNORB_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: category **********
Accuracy: 0.9967527060782682
********** Model: instance **********
Accuracy: 0.8997502081598667
********** Model: elevation **********
Accuracy: 0.504496253122398
********** Model: azimuth **********
Accuracy: 0.8284762697751873
********** Model: lighting **********
Accuracy: 0.9209825145711906

******************** medium ********************
********** Model: category **********
Accuracy: 0.9971184072651065
********** Model: instance **********
Accuracy: 0.9047327977645826
********** Model: elevation **********
Accuracy: 0.49048201187565493
********** Model: azimuth **********
Accuracy: 0.8359238560950052
********** Model: lighting **********
Accuracy: 0.9242926999650716

******************** large ********************
********** Model: category **********
Accuracy: 0.9975822949600148
********** Model: instance **********
Accuracy: 0.9145434257020644
********** Model: elevation **********
Accuracy: 

## Image shift

In [20]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_img = []

for shift_intensity in shift_intensities:
    X_shifted = deepcopy(X_test_flatten)
    X_shifted, _ = apply_img_shift(X_shifted, y_test, ORIGINAL_SHAPE,
                                   shift_intensity, 1.0, ShiftType.All)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_img.append(X_shifted)

In [21]:
# Print accuracy score (end-to-end)
for x, intensity_str in zip(X_img, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

******************** small ********************
Accuracy: 0.8819753086419753
******************** medium ********************
Accuracy: 0.3493827160493827
******************** large ********************
Accuracy: 0.22493827160493826


In [22]:
# Print accuracy score (cbm)
for x, intensity_str in zip(X_img, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SMALLNORB_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c_test[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: category **********
Accuracy: 0.8693004115226337
********** Model: instance **********
Accuracy: 0.397037037037037
********** Model: elevation **********
Accuracy: 0.2702880658436214
********** Model: azimuth **********
Accuracy: 0.29218106995884774
********** Model: lighting **********
Accuracy: 0.8089711934156378

******************** medium ********************
********** Model: category **********
Accuracy: 0.357119341563786
********** Model: instance **********
Accuracy: 0.14189300411522635
********** Model: elevation **********
Accuracy: 0.14386831275720163
********** Model: azimuth **********
Accuracy: 0.06510288065843621
********** Model: lighting **********
Accuracy: 0.4918518518518519

******************** large ********************
********** Model: category **********
Accuracy: 0.21761316872427983
********** Model: instance **********
Accuracy: 0.10880658436213991
********** Model: elevation **********
Accura

## Adversarial shift

In [23]:
shift_intensities = [ShiftIntensity.Small, ShiftIntensity.Medium, ShiftIntensity.Large]
X_adv = []

for shift_intensity in shift_intensities:
    X_shifted = deepcopy(X_test_flatten)
    X_shifted, _ = apply_adversarial_shift(X_shifted, y_test, adv_samples,
                                   range(len(X_shifted)), shift_intensity)
    X_shifted = X_shifted.reshape(-1, ORIGINAL_SHAPE[0], ORIGINAL_SHAPE[1], ORIGINAL_SHAPE[2])
    X_adv.append(X_shifted)

In [24]:
# Print accuracy score (end-to-end)
for x, intensity_str in zip(X_adv, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    y_pred = end_to_end_model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")

******************** small ********************
Accuracy: 0.9734979423868313
******************** medium ********************
Accuracy: 0.8804938271604938
******************** large ********************
Accuracy: 0.7604115226337449


In [25]:
# Print accuracy score (cbm)
for x, intensity_str in zip(X_adv, ["small", "medium", "large"]):
    print("*"*20, intensity_str, "*"*20)
    for i, pred in enumerate(itc_model.predict(x)):
        print("*"*10, f"Model: {SMALLNORB_CONCEPT_NAMES[i]}", "*"*10)
        c_truth = c_test[:, i]
        c_pred = np.argmax(pred, axis=1)

        print(f"Accuracy: {accuracy_score(c_truth, c_pred)}")
    print()

******************** small ********************
********** Model: category **********
Accuracy: 0.9653497942386832
********** Model: instance **********
Accuracy: 0.8465020576131688
********** Model: elevation **********
Accuracy: 0.47802469135802467
********** Model: azimuth **********
Accuracy: 0.7731687242798354
********** Model: lighting **********
Accuracy: 0.8690534979423868

******************** medium ********************
********** Model: category **********
Accuracy: 0.8422222222222222
********** Model: instance **********
Accuracy: 0.6465843621399177
********** Model: elevation **********
Accuracy: 0.36493827160493825
********** Model: azimuth **********
Accuracy: 0.560082304526749
********** Model: lighting **********
Accuracy: 0.6516049382716049

******************** large ********************
********** Model: category **********
Accuracy: 0.6851851851851852
********** Model: instance **********
Accuracy: 0.39209876543209876
********** Model: elevation **********
Accuracy